### Scrape data
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import snscrape.modules.twitter as sntwitter
import seaborn as sns

from nltk.corpus import stopwords, words # get stopwords from NLTK library & get all words in english language
from nltk.tokenize import word_tokenize # to create word tokens
# from nltk.stem import PorterStemmer (I played around with Stemmer and decided to use Lemmatizer instead)
from nltk.stem import WordNetLemmatizer # to reduce words to orginal form
from nltk import pos_tag # For Parts of Speech tagging

In [2]:
query = '(FIFA World Cup OR FIFA World Cup 2022 OR FIFA World Cup 2022 Qatar OR #FIFAWorldCup OR #FIFAWorldCup2022 OR #FIFAWorldCup2022Qatar)until:2022-12-05 since:2022-01-01'

tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i > 20000:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.user.location,
                       tweet.content, tweet.likeCount, tweet.retweetCount])
df = pd.DataFrame(tweets, columns= ['Date', 'ID', 'url', 'user_name', 'location', 'content', 'likes', 'retweet'])
df.to_csv('twitter.csv', mode='a')  

In [ ]:
df1 = pd.read_csv('/home/miki/Desktop/Projects/Hamoye/twitter-sentiment-analysis/twitter.csv', encoding='unicode_escape')
df1.head(3)

,Unnamed: 0,Date,ID,url,user_name,location,content,likes,retweet
